# 1.0 Data Collection (Web Crawling)

###### Author: Terence Tiu Chuan Jie 
###### Last Edited:9/4/2025


## 1.Newsdata io

In [2]:
from pyspark.sql import SparkSession

import sys

sys.path.append(r'/home/student/data_collected')


from newsdataapi import NewsDataApiClient
import pickle
import csv  

spark = SparkSession.builder.appName('Newsdata io').getOrCreate()

25/04/09 22:59:27 WARN Utils: Your hostname, tiu. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/09 22:59:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/09 22:59:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/09 22:59:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:

api = NewsDataApiClient(apikey="pub_")


In [4]:
import csv

class NewsFetcher:
    def __init__(self, api, country='us', category='crime', language='en'):
        self.api = api
        self.country = country
        self.category = category
        self.language = language
        self.news_list = []

    def fetch_all_news(self):
        page = None
        while True:
            try:
                response = self.api.news_api(
                    country=self.country,
                    category=self.category,
                    language=self.language,
                    page=page
                )
                news = response.get('results', [])
                self.news_list.extend(news)

                page = response.get('nextPage', None)
                if not page:
                    print("No more pages available.")
                    break
            except Exception as e:
                print(f"Error: {e}")
                break

    def get_news(self):
        return self.news_list


class NewsAnalyzer:
    def __init__(self, news_list):
        self.news_list = news_list

    def display_sample_news(self, count=5):
        for i, news in enumerate(self.news_list[:count]):
            print(f"News {i}: {news}")

    def get_all_keys(self):
        all_keys = set()
        for news in self.news_list:
            all_keys.update(news.keys())
        return all_keys


class NewsExporter:
    def __init__(self, news_list, all_keys):
        self.news_list = news_list
        self.all_keys = list(all_keys)

    def export_to_csv(self, filename='newsdata_io_9_4_test.csv'):
        try:
            with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=self.all_keys)
                writer.writeheader()
                for news in self.news_list:
                    writer.writerow(news)
            print(f"Exported {len(self.news_list)} articles to {filename}")
        except Exception as e:
            print(f"Failed to write CSV: {e}")


# === Usage ===
# `api` is already defined and connected

fetcher = NewsFetcher(api)
fetcher.fetch_all_news()

news_list = fetcher.get_news()
print(f"Total articles retrieved: {len(news_list)}")

analyzer = NewsAnalyzer(news_list)
analyzer.display_sample_news()
all_keys = analyzer.get_all_keys()
print("All keys found in articles:", all_keys)

exporter = NewsExporter(news_list, all_keys)
exporter.export_to_csv()


/tmp/ipykernel_29599/2805961141.py:15: DeprecationWarning: This method is deprecated and will be removed in upcoming updates, Instead use latest_api()
  response = self.api.news_api(


Error: {'status': 'error', 'results': {'message': 'You exceeded your assigned API credits, please check your plan and billing details.', 'code': 'ApiLimitExceeded'}}
Total articles retrieved: 190
News 0: {'article_id': '7cf5fadd91226f2920449067099412b0', 'title': 'Public Safety Logs — Monday, April 7, 2025', 'link': 'https://www.romesentinel.com/news/public-safety/public-safety-logs-monday-april-7-2025/article_fe6ead05-7304-4aa5-bc11-ba6926b5a761.html', 'keywords': None, 'creator': None, 'description': 'Rome Fire Department log', 'content': 'ONLY AVAILABLE IN PAID PLANS', 'pubDate': '2025-04-09 02:00:00', 'pubDateTZ': 'UTC', 'image_url': None, 'video_url': None, 'source_id': 'romesentinel', 'source_name': 'Romesentinel', 'source_priority': 38985, 'source_url': 'https://www.romesentinel.com', 'source_icon': 'https://i.bytvi.com/domain_icons/romesentinel.jpg', 'language': 'english', 'country': ['united states of america'], 'category': ['crime'], 'sentiment': 'ONLY AVAILABLE IN PROFESSION

In [5]:
spark.stop()

In [ ]:
## if u wanna sue for pre- preprocessing for here some sample ideal 
import json

for news in news_list:
    for key, value in news.items():
        if isinstance(value, list):
            news[key] = ', '.join(value) if value else None  # Convert list to string
        elif isinstance(value, dict):
            news[key] = json.dumps(value)  # Convert dict to JSON string
